In [1]:
import numpy as np
import pandas as pd

## Punto 1

In [2]:
# before normalizing the loan_lenders.csv table, let's check that each username
# doesn't contain spaces or commas
data_folder_path = 'additional-kiva-snapshot/'
lenders = pd.read_csv(data_folder_path + 'lenders.csv')
pd.set_option('display.max_rows', 10)
lenders

,permanent_name,display_name,city,state,country_code,member_since,occupation,loan_because,loan_purchase_num,invited_by,num_invited
0,qian3013,Qian,NaN,NaN,NaN,1461300457,NaN,NaN,1.0,NaN,0
1,reena6733,Reena,NaN,NaN,NaN,1461300634,NaN,NaN,9.0,NaN,0
2,mai5982,Mai,NaN,NaN,NaN,1461300853,NaN,NaN,NaN,NaN,0
3,andrew86079135,Andrew,NaN,NaN,NaN,1461301091,NaN,NaN,5.0,Peter Tan,0
4,nguyen6962,Nguyen,NaN,NaN,NaN,1461301154,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
2349169,janet7309,Janet,NaN,NaN,NaN,1342097163,NaN,NaN,NaN,NaN,0
2349170,pj4198,NaN,NaN,NaN,NaN,1342097515,NaN,NaN,NaN,NaN,0
2349171,maria2141,Maria,NaN,NaN,US,1342099723,NaN,NaN,2.0,NaN,0
2349172,simone9846,Simone,NaN,NaN,NaN,1342100213,NaN,NaN,NaN,NaN,0


In [3]:
space = lenders['permanent_name'].str.find(' ') == -1
comma = lenders['permanent_name'].str.find(',') == -1
total = space & comma
del space, comma
assert np.any(total), 'Invalid account name'
del total

In [4]:
loan_lenders = pd.read_csv(data_folder_path + 'loans_lenders.csv')
loan_lenders

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."
...,...,...
1387427,678999,"michael43411218, carol5987, gooddogg1, chris41..."
1387428,1207353,"rjhoward1986, jeffrey6870, trolltech4460, elys..."
1387429,1206220,"vicky7746, gooddogg1, fairspirit, craig9729960..."
1387430,1206425,"rich6705, sergiiy9766, angela7509, barbara5610..."


In [5]:
loan_lenders['lenders'][0:10].str.split(', ', expand = True)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,muc888,sam4326,camaran3922,lachheb1865,rebecca3499,karlheinz4543,jerrydb,paula8951,gmct,amra9383,...,jonathan7946,ann8187,bryan2669,john88459657,eddyphil,don9212,carolineandcolin9686,bent8782,raph8817,danielle2350
1,muc888,nora3555,williammanashi,barbara5610,danhostetler,danhostetler,danhostetler,daniel1104,amirali5409,oceanwest,...,None,None,None,None,None,None,None,None,None,None
2,muc888,terrystl,richardandsusan8352,sherri4341,mattiaslaven,shree8053,gooddogg1,don9212,raph8817,danny6470,...,None,None,None,None,None,None,None,None,None,None
3,muc888,james5068,rudi5955,daniel9859,don9212,chandrakant6134,karyne5301,robert2280,darrellynne7000,bryan9912,...,None,None,None,None,None,None,None,None,None,None
4,muc888,niki3008,teresa9174,mike4896,david7683,dittes,margaret7172,jessica1977,william7288,gooddogg1,...,None,None,None,None,None,None,None,None,None,None
5,muc888,tristan7990,shivaun4955,sam44598568,mike4896,catherine7003,summer7416,jim8842,brett5260,roger2252,...,None,None,None,None,None,None,None,None,None,None
6,muc888,john38425073,trolltech4460,marianne8717,lisette1963,motoharu1020,motoharu1020,gooddogg1,elly3162,lena9976,...,None,None,None,None,None,None,None,None,None,None
7,muc888,dougal1825,dougal1825,jensdamsgaardvanar,maeve9455,ross6632,toni3909,ali3337,eric4693,mike4896,...,None,None,None,None,None,None,None,None,None,None
8,muc888,rebecca3038,paul1853,paul1853,paul1853,zach7677,dana9865,dana9865,ankit3005,erich7889,...,None,None,None,None,None,None,None,None,None,None
9,klaus5005,john70242429,john70242429,terry9343,karly7529,donna1305,jodi8604,jitendra7163,leon2765,elizabeth8018,...,alexander7609,rc4114,None,None,None,None,None,None,None,None
